In [16]:
#import libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

#list of years to collect
years = range(2012, 2022)

#url with coach W/L data
base_url = 'https://www.sports-reference.com/cbb/seasons/{}-coaches.html'

#creating empty dataframe
coaches = pd.DataFrame()

#for loop to scrape coaching data and append into one dataframe
for year in years:
    req_url = base_url.format(year)
    req = requests.get(req_url)
    soup = BeautifulSoup(req.content, 'html.parser')
    table = soup.find('table', {'id':'coaches'})
    df = pd.read_html(str(table))[0]
    df.columns = df.columns.droplevel()
    df['Year'] = year
    coaches = coaches.append(df)

#function to remove duplicate column names and only keep the first W and L columns (these are the total wins and losses from regular season and tourney)
def remove_dup_columns(frame):
     keep_names = set()
     keep_icols = list()
     for icol, name in enumerate(frame.columns):
          if name not in keep_names:
               keep_names.add(name)
               keep_icols.append(icol)
     return frame.iloc[:, keep_icols]

#call function on coaching data
coaches = remove_dup_columns(coaches)

#create new dataframe with necessary columns
new = coaches[['Coach','School','W','L','Year']]

#make sure all names are consistent, names with * indicates tourney apperances but this breaks the same coaches into two separate names so need to remove this and the white space left behind after removing
new['Coach'] = new['Coach'].str.rstrip('*')
new['Coach'] = new['Coach'].str.strip()

#drop blank rows
new = new.dropna()

#rename columns to drop placeholder rows
new.columns = ['Coach_Name', 'School_Name', 'Wins', 'Losses','Season']
new = new[new.Coach_Name != 'Coach']

#convert datatypes for pivot table
new = new.astype({"Wins": int, "Losses": int})

#create pivot table like excel to sum wins and losses
table = pd.pivot_table(new, index=['Coach_Name'], values= ('Wins', 'Losses'),
                     aggfunc=['sum'])

#drop index row. reorder columns
table.columns = table.columns.droplevel()
table = table[['Wins', 'Losses']]

#create all time winning percentage column and calculate
table['W_L%'] = (table['Wins'].astype(int)/ (table['Wins'].astype(int) + table['Losses'].astype(int))).round(2)

#sort table by total wins and show top 15
table = table.sort_values(by='Wins',ascending=False)
print('Winningest Coaches over past 10 seasons by Total Wins')
table.head(15)

Winningest Coaches over past 10 seasons by Total Wins


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Wins,Losses,W_L%
Coach_Name,,,
Mark Few,314,42,0.88
Bill Self,285,72,0.80
John Calipari,275,81,0.77
Mike Krzyzewski,270,77,0.78
Jay Wright,266,78,0.77
Tony Bennett,264,72,0.79
Roy Williams,260,101,0.72
Tom Izzo,260,93,0.74
Dana Altman,259,92,0.74


In [17]:
#Remove outlier who only coached one game and won
i = table[((table.index == 'Grant Billmeier'))].index
table = table.drop(i)

#sort table by win% and show top 15
print('Winningest Coaches over past 10 seasons by Win %')
table = table.sort_values(by='W_L%',ascending=False)
table.head(15)

Winningest Coaches over past 10 seasons by Win %


,Wins,Losses,W_L%
Coach_Name,,,
Mark Few,314,42,0.88
Bill Self,285,72,0.80
Tony Bennett,264,72,0.79
Gregg Marshall,249,66,0.79
Rick Pitino,183,53,0.78
Mike Krzyzewski,270,77,0.78
John Calipari,275,81,0.77
Jay Wright,266,78,0.77
Brian Dutcher,96,31,0.76
